In [14]:
import ROOT
ROOT.PyConfig.IgnoreCommandLineOptions = True
ROOT.gROOT.SetBatch(True)
ROOT.gSystem.Load("libFWCoreFWLite.so");
ROOT.gSystem.Load("libDataFormatsFWLite.so");
ROOT.FWLiteEnabler.enable()

In [15]:
import sys
from math import sqrt, cosh
from DataFormats.FWLite import Handle, Events
from PhysicsTools.HeppyCore.utils.deltar import deltaR, matchObjectCollection, matchObjectCollection3, bestMatch
import numpy as np
import pandas as pd

In [16]:
#helper functions
def close_to(v1, ref):
    return abs(v1 - ref) < 0.0005*abs(v1+ref) #1 per mil of scale

def kin_close(obj, ref):
    pt, eta, phi = ref
    return close_to(obj.pt(), pt) and close_to(obj.eta(), eta) and close_to(obj.phi(), phi)

def print_kin(obj):
    print '%s[%f, %f, %f]' % (obj.__class__.__name__, obj.pt(), obj.eta(), obj.phi())
    
#copied from https://github.com/cms-sw/cmssw/blob/master/RecoParticleFlow/PFTracking/src/PFTrackAlgoTools.cc#L170
not_fifth = {
    'ctf', 'duplicateMerge', 'initialStep', 'highPtTripletStep', 'lowPtQuadStep', 'lowPtTripletStep', 'pixelPairStep',
    'jetCoreRegionalStep', 'detachedQuadStep', 'detachedTripletStep', 'mixedTripletStep', 'pixelLessStep', 
    'muonSeededStepInOut', 'muonSeededStepOutIn', 'hltIter0', 'hltIter1', 'hltIter2', 'hltIter3', 'hltIter4', 
    'hltIterX',
}
def is_fifth_step(algo):
    return (TrkAlgos[algo] not in not_fifth)

In [17]:
htrk = Handle("std::vector<reco::Track>")
hvtx = Handle("std::vector<reco::Vertex>")
hpf = Handle("std::vector<reco::PFCandidate>")
hpfb = Handle("std::vector<reco::PFBlock>")
hgsfe = Handle("std::vector<reco::GsfElectron>")
hpf_ele_trk = Handle('vector<reco::GsfPFRecTrack>')
#hmu   = Handle("std::vector<reco::Muon>")
hpfc = Handle("std::vector<reco::PFCluster>")
#hcalt = Handle("edm::SortedCollection<CaloTower,edm::StrictWeakOrdering<CaloTower> >")
genp  = Handle("std::vector<reco::GenParticle>")
hgsf_trk = Handle('vector<reco::GsfTrack>')

In [18]:
h_pf_trk = Handle('vector<reco::PFRecTrack>')


In [19]:
PFTypes = {}
for tn in "NONE TRACK PS1 PS2 ECAL HCAL GSF BREM HFEM HFHAD SC HO HGCAL kNBETypes".split():
    PFTypes[tn] = getattr(ROOT.reco.PFBlockElement, tn)
    PFTypes[PFTypes[tn]] = tn
    
TrkAlgos = {}
for al in ('undefAlgorithm ctf duplicateMerge cosmics initialStep lowPtTripletStep pixelPairStep '
           'detachedTripletStep mixedTripletStep pixelLessStep tobTecStep jetCoreRegionalStep '
           'conversionStep muonSeededStepInOut muonSeededStepOutIn outInEcalSeededConv inOutEcalSeededConv '
           'nuclInter standAloneMuon globalMuon cosmicStandAloneMuon cosmicGlobalMuon highPtTripletStep lowPtQuadStep '
           'detachedQuadStep reservedForUpgrades1 reservedForUpgrades2 bTagGhostTracks beamhalo gsf hltPixel hltIter0 '
           'hltIter1 hltIter2 hltIter3 hltIter4 hltIterX hiRegitMuInitialStep hiRegitMuLowPtTripletStep '
           'hiRegitMuPixelPairStep hiRegitMuDetachedTripletStep hiRegitMuMixedTripletStep hiRegitMuPixelLessStep '
           'hiRegitMuTobTecStep hiRegitMuMuonSeededStepInOut hiRegitMuMuonSeededStepOutIn algoSize').split():
        TrkAlgos[al] = getattr(ROOT.reco.TrackBase, al)
        TrkAlgos[TrkAlgos[al]] = al

In [20]:
events = Events('EDOutput.root')
iterator = events.__iter__()

In [21]:

#copied from the ntuple
gsf_pt_eta_phi = {}

#with PFGSF, but no PFEgamma
wPFGSF_No_PFEG = {60624397: (3.2524049282073975, 0.37861114740371704, -2.5944802761077881), 59624562: (1.3961746692657471, -1.7653611898422241, 0.35166314244270325), 61035956: (1.3295170068740845, -0.74432915449142456, -0.89448297023773193), 56535285: (2.057009220123291, -0.78594261407852173, -2.8598525524139404), 59274487: (1.2078419923782349, -0.71046125888824463, 1.0331467390060425), 61399352: (1.9714157581329346, -1.4073941707611084, 1.7577246427536011), 60333114: (1.0912861824035645, -0.87859433889389038, -2.2439939975738525), 61752053: (1.9641132354736328, -1.4127223491668701, 2.0960731506347656)}
wGSFTrk_No_PFGSF = {59472208: (4.6232380867004395, -0.65895694494247437, -0.25869864225387573), 60321251: (1.1779265403747559, 0.82428324222564697, 1.5455923080444336), 61381254: (4.7505941390991211, 0.25948464870452881, -2.0914435386657715), 58402401: (1.7245339155197144, 0.44584566354751587, -1.5300849676132202), 57817001: (9.2789793014526367, 2.2519600391387939, 0.67897021770477295), 61743594: (9.8303279876708984, 0.39349210262298584, 2.9180049896240234), 58865712: (2.4383711814880371, -1.5381476879119873, 3.0827889442443848), 61785330: (1.6721429824829102, -0.74857711791992188, -1.1605315208435059), 57160055: (2.4560666084289551, 0.96155643463134766, -2.9944767951965332), 61091548: (17.432041168212891, -0.23632617294788361, 2.0985651016235352), 31949309: (3.1733708381652832, 1.6469196081161499, -1.1084054708480835)}


In [22]:
evt = iterator.next()
run, lumi, evtno = evt.eventAuxiliary().run(), evt.eventAuxiliary().luminosityBlock(), evt.eventAuxiliary().event()
print ':'.join([str(i) for i in [run, lumi, evtno]])
if evtno in wPFGSF_No_PFEG:
    print 'HAS PFGSF, no PF EGamma'
    gsf_pt, gsf_eta, gsf_phi = wPFGSF_No_PFEG[evtno]
elif evtno in wGSFTrk_No_PFGSF:
    print 'HAS GSFTrack, no PFGSF'
    gsf_pt, gsf_eta, gsf_phi = wGSFTrk_No_PFGSF[evtno]

1:106498:31949309
HAS GSFTrack, no PFGSF


## GSF Matching and PFRecGSFTrack Matching
Find the GSF track and its index, might come handy

In [23]:
evt.getByLabel('electronGsfTracks', hgsf_trk)
gsf_tracks = hgsf_trk.product()
matches = [kin_close(i, (gsf_pt, gsf_eta, gsf_phi)) for i in gsf_tracks]
if sum(matches) > 1: print 'MULTIPLE MATCHES!'
elif sum(matches) == 0: print 'NO MATCH?!'
else:
    print 'MATCHED!'
    gsf_idx = matches.index(1)
    matched_gsf = gsf_tracks[gsf_idx]

MATCHED!


Check that there is a PFRecGSFTrack

In [24]:
evt.getByLabel('pfTrackElec', hpf_ele_trk)
pf_gsf_tracks = hpf_ele_trk.product()
gsf_indexes = [i.gsfTrackRef().index() for i in pf_gsf_tracks]
if gsf_idx in gsf_indexes:
    print 'MATCH!'
    pf_gsf_idx = gsf_indexes.index(gsf_idx)
    matched_pf_gsf = pf_gsf_tracks[pf_gsf_idx]
else:
    print 'NO MATCH!'

MATCH!


#### PFRecGSFTrack Debugging

In [12]:
evt.getByLabel('pfTrack', h_pf_trk)
pf_trks = h_pf_trk.product()

Assumes that everything exists here

In [18]:
ele_seed = matched_gsf.seedRef().get()
matched_pf_track = [i for i in pf_trks if i.trackRef() == ele_seed.ctfTrack()]

In [20]:
if matched_pf_track:
    print 'MATCHED TO PFTrack'
    matched_pf_track = matched_pf_track[0]
else:
    print 'NO MATCH'

MATCHED TO PFTrack


In [26]:
track_attributes = set()
if matched_gsf.seedRef().caloCluster().isNonnull():
    track_attributes.add('ECALDriven')
if matched_gsf.seedRef().ctfTrack().isNonnull():
    track_attributes.add('TrakerDriven')
if is_fifth_step(matched_pf_track.trackRef().algo()):
    track_attributes.add('FifthStep')
print track_attributes

set(['TrakerDriven'])


In [ ]:
nPin = matched_gsf.pMode()
neta = matched_gsf.innerMomentum().eta()
nphi = matched_gsf.innerMomentum().phi()

all_Pin = [i.pMode() for i in gsf_tracks]
all_eta = [i.pMode() for i in gsf_tracks]
all_phi = [i.pMode() for i in gsf_tracks]

all_Pin.pop(gsf_idx)
all_eta.pop(gsf_idx)
all_phi.pop(gsf_idx)

all_Pin = np.array(all_Pin)
all_eta = np.array(all_eta)
all_phi = np.array(all_phi)

delta_eta = neta - all_eta
delta_phi = nphi - all_phi

In [26]:
evt.getByLabel('particleFlowClusterECAL', hpfc)
pf_clusters = hpfc.product()

In [28]:
[i.calculatePositionREP() for i in pf_clusters]
close_clusters = [i for i in pf_clusters 
                  if abs(i.position().eta() - matched_gsf.eta()) < 0.5
                  if abs(i.position().phi() - matched_gsf.phi()) < 1]
print_kin(matched_gsf)
print_kin(matched_pf_gsf.gsfTrackRef().get())
print matched_gsf.outerPt()
for i in close_clusters:
    print_kin(i)

GsfTrack[3.173371, 1.646920, -1.108405]
GsfTrack[3.173371, 1.646920, -1.108405]
0.275696496951
PFCluster[1.166230, 1.647960, -0.968813]
PFCluster[0.654668, 2.139844, -0.419574]
PFCluster[0.730601, 2.005828, -1.552286]
PFCluster[0.412121, 1.871694, -0.995318]
PFCluster[0.361261, 1.303286, -1.411212]
PFCluster[0.225871, 1.218185, -0.459580]
PFCluster[0.145714, 1.423141, -1.700597]
PFCluster[0.271246, 1.354865, -1.812954]
PFCluster[0.126335, 1.474612, -1.980674]
PFCluster[0.124045, 1.355571, -0.183568]
PFCluster[0.136011, 1.235496, -1.506889]
PFCluster[0.108817, 1.475371, -1.718014]


In [31]:
ECAL_position = matched_pf_gsf.extrapolatedPoint( ROOT.reco.PFTrajectoryPoint.ECALShowerMax )
ECAL_position.positionREP().Eta(), ECAL_position.positionREP().Phi()

(1.9897250368800312, -0.4586583776130577)

In [17]:
other_gsf = [i for i in gsf_tracks if abs(i.pt() - 2.717) < 0.01][0]
close_clusters = [i for i in pf_clusters 
                  if abs(i.position().eta() - other_gsf.eta()) < 0.5
                  if abs(i.position().phi() - other_gsf.phi()) < 1]
print_kin(other_gsf)
print other_gsf.outerPt()
for i in close_clusters:
    print_kin(i)

GsfTrack[2.717000, 1.708730, -1.274414]
0.703981300744
PFCluster[1.166230, 1.647960, -0.968813]
PFCluster[0.654668, 2.139844, -0.419574]
PFCluster[0.730601, 2.005828, -1.552286]
PFCluster[0.412121, 1.871694, -0.995318]
PFCluster[0.361261, 1.303286, -1.411212]
PFCluster[0.225871, 1.218185, -0.459580]
PFCluster[0.145714, 1.423141, -1.700597]
PFCluster[0.271246, 1.354865, -1.812954]
PFCluster[0.126335, 1.474612, -1.980674]
PFCluster[0.136011, 1.235496, -1.506889]
PFCluster[0.108817, 1.475371, -1.718014]


False

In [68]:
evt.getByLabel('particleFlowBlock', hpfb)
blocks = hpfb.product()

In [69]:
matched_block = None
matched_element = None
for block in blocks:
    for element in block.elements():
        if element.type() != PFTypes['GSF']: continue
        ref_idx = element.GsftrackRef().index()
        if ref_idx == gsf_idx:
            if matched_block is None:
                print 'MATCH!'
                matched_block = block
                matched_element = element
            else:
                print 'MULTIPLE MATCHES!'

MATCH!


In [70]:
for el in matched_block.elements():
    print PFTypes[el.type()]

ecal = [i for i in matched_block.elements() if i.type() == PFTypes['ECAL']][0]


BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
BREM
GSF
TRACK


IndexError: list index out of range

In [56]:
print ecal.clusterRef().get().energy()
matched_gsf.outerP()

0.886527001858


2.372606438630701

In [57]:
matched_element

<ROOT.reco::PFBlockElementGsfTrack object at 0x13d10600>

In [12]:
evt.getByLabel('particleFlowEGamma', hpf)
pfeg = hpf.product()

In [13]:
pf_gsfs = [i for i in pfeg if not i.gsfTrackRef().isNull() and close_to(i.gsfTrackRef().pt(), gsf_pt)]

In [14]:
print_kin(pf_gsfs[0].gsfTrackRef().get())
gsf_pt_eta_phi[evtno]

IndexError: list index out of range

In [15]:
pf_gsfs

[]